In [3]:
import networkx as nx
import math
import pickle

In [ ]:
# Read a directed graph
graph = pickle.load('graph.p')

In [16]:
# Calculate feature values of each node based on the set of metrics we use
# 1. Number of followers
# 2. Number of circles you are a part of in other people's circles
# 3. Sum of fractions of circles that you are a part of
# 4. Average of fractions of circles that you are a part of

def calculateFeatureValues(graph):
    for node, data in graph.nodes_iter(data=True):
        in_edges = graph.in_edges(data=True)
        total_circles = 0
        total_fraction = 0
        for n1, n2, edata in in_edges:
            total_circles += edata['circles']
            total_fraction += (edata['circles'] * 1.0 / graph.node[n1]['circles'])
        
        average_fraction = 0
        if len(in_edges) > 0:
            average_fraction = total_fraction / in_edges
        
        data['followers'] = len(in_edges)
        data['in_circles'] = total_circles
        data['sum_circle_fraction'] = total_fraction
        data['avg_circle_fraction'] = total_fraction
    
        print gr.node[node]

In [20]:
# Feature weights
feature_weights = {}
feature_weights['followers'] = 1
feature_weights['in_circles'] = 1
feature_weights['sum_circle_fraction'] = 1
feature_weights['avg_circle_fraction'] = 1

In [29]:
# Calculating the score for every node based on our metric
def calculateScores(graph):
    scores = {}
    for node, data in graph.nodes_iter(data=True):
        score = 0
        for feature in feature_weights:
            score += feature_weights[feature] * data[feature]
        
        scores[node] = score
        print "Node", node, "with gender", graph.node[node]['gender'], "had a score of", score, "using our metric"
        
    return scores

In [30]:
gr = nx.DiGraph()
gr.add_node("test",  gender='M')
calculateFeatureValues(gr)
scores = calculateScores(gr)
print scores

{'gender': 'M', 'sum_circle_fraction': 0, 'avg_circle_fraction': 0, 'followers': 0, 'in_circles': 0}
Node test with gender M had a score of 0 using our metric
{'test': 0}
